In [1]:
import pandas as pd

pd.options.display.max_columns = 500

In [2]:
# import data
df = pd.read_csv('base_congresso.csv')

df.head()

,id,vintage,internal_feature_0,internal_feature_1,internal_feature_2,internal_feature_3,internal_feature_4,internal_feature_5,internal_feature_6,internal_feature_7,internal_feature_8,internal_feature_9,internal_feature_10,internal_feature_11,bacen_feature_0,bacen_feature_1,bacen_feature_2,bacen_feature_3,bacen_feature_4,bacen_feature_5,bacen_feature_6,bacen_feature_7,bacen_feature_8,bacen_feature_9,bacen_feature_10,bacen_feature_11,bacen_feature_12,bacen_feature_13,bacen_feature_14,bacen_feature_15,bacen_feature_16,bacen_feature_17,bureau_feature_0,bureau_feature_1,bureau_feature_2,bureau_feature_3,bureau_feature_4,bureau_feature_5,bureau_feature_6,bureau_feature_7,bureau_feature_8,bureau_feature_9,bureau_feature_10,bureau_feature_11,bureau_feature_12,bureau_feature_13,bureau_feature_14,bureau_feature_15,bureau_feature_16,bureau_feature_17,bureau_feature_18,bureau_feature_19,bureau_feature_20,bureau_feature_21,installments,is_loan,is_default
0,10751409,2021-03,WEB,4000.0,899.0,L5,REGISTERED_PRIVATE,PCRE,2000.0,93.32,27,SINGLE,POST_GRADUATION,RS,0.0,111995.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111995.0,0.0,0.0,0.0,0.0,128457.0,0.0,NaN,0.0,0.0,0.0,1.0,3.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,5.0,POSSUI,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,12,1,0
1,11544139,2021-08,WEB,5000.0,856.0,L6,REGISTERED_PRIVATE,ELET,4000.0,58.27,35,SINGLE,POST_GRADUATION,MG,0.0,0.0,4527.0,65397.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69924.0,0.0,0.0,0.0,0.0,2662141.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,3.0,POSSUI,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,18,0,0
2,9208477,2020-02,serasa-ecred,2737.2,607.0,L8,REGISTERED_PRIVATE,PDIV,3205.0,126.21,27,SINGLE,HIGH_SCHOOL,ES,0.0,0.0,0.0,151457.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,151457.0,0.0,0.0,0.0,0.0,114.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,1,0
3,10219088,2020-11,FinanZero,2600.0,640.0,L8,REGISTERED_PRIVATE,WEDD,7000.0,101.24,32,MARRIED,COLLEGE_EDUCATION_INCOMPLETE,SP,1716413.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,121491.0,121491.0,1716413.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,1,1
4,11082451,2021-06,serasa-ecred,3000.0,924.0,L3,REGISTERED_PRIVATE,PDIV,2500.0,4.67,26,MARRIED,COLLEGE_EDUCATION_INCOMPLETE,PR,0.0,0.0,0.0,5608.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5608.0,0.0,0.0,0.0,0.0,287096.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,1.0,NAO_POSSUI,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,12,1,0


In [3]:
# Create relevance score column
df['relevance_score'] = df[['is_loan', 'is_default']].apply(lambda row: row[0]+(1-row[1]), axis = 1)

In [4]:
df['vintage'] = df['vintage'].apply(pd.to_datetime)

In [5]:
df_train = df.loc[df['vintage'] <= '2021-06-01'].copy()
df_val = df.loc[df['vintage'] > '2021-06-01'].copy()

In [6]:
# Check null values percentage by column
nulls = (df_train.isnull().sum()/df.shape[0]) \
            .sort_values(ascending = False) \
            .reset_index() \
            .rename(columns = {'index': 'column_name', 0: 'null_perc'})

nulls_list = nulls.loc[nulls['null_perc'] > 0.5, 'column_name'].tolist()

In [7]:
# Delete features with more than 50% null values
df_train.drop(columns = nulls_list, inplace = True)
df_val.drop(columns = nulls_list, inplace = True)

In [8]:
# Separate features
keys = ['id', 'vintage']
ranking_target = ['installments']
score_composition = ['is_loan', 'is_default', 'relevance_score']
features = [col for col in df_train.columns if col not in keys + ranking_target + score_composition]

In [9]:
# Treatment of null values for internal categorical and numerical features
internal_feat_mode = df_train.filter(regex = ('internal_feature*')) \
                           .mode() \
                           .to_dict('list')

for key in internal_feat_mode:
    df_train[key].fillna(internal_feat_mode[key][0], inplace = True)
    df_val[key].fillna(internal_feat_mode[key][0], inplace = True)

In [10]:
# Treatment of null values for external categorical features
external_cat_feat_mode = df_train.filter(regex = ('^((?!internal_feature).)*$')) \
                           .select_dtypes('object') \
                           .columns \
                           .to_list()

for key in external_cat_feat_mode:
    df_train[key].fillna('Not_found', inplace = True)
    df_val[key].fillna('Not_found', inplace = True)

In [11]:
# Treatment of null values for external numerical features
external_num_feat_mode = df_train.filter(regex = ('^((?!internal_feature).)*$')) \
                           .select_dtypes(exclude = 'object') \
                           .columns \
                           .to_list()

for key in external_num_feat_mode:
    df_train[key].fillna(0, inplace = True)
    df_val[key].fillna(0, inplace = True)

In [ ]:
df_train.to_csv('train_data_nonull.csv', index = False)
df_train.to_csv('train_data_nonull.csv', index = False)